In [1]:
import pandas as pd
import numpy as np
import wrds

In [2]:
from datetime import datetime
from scipy.stats import skew, kurtosis
import time

In [4]:
# Establish connection to WRDS
db = wrds.Connection()

Enter your WRDS username [ec2-user]: ly229
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


## Ownership data of Fund Holdings(index, active, and institutional)

#### check tables under certain library(e.g Thomson-Reuters, CRSP) in wrds to find the correct name of database used

In [3]:
# List available tables in Thomson Reuters dataset
tables = db.list_tables(library="tfn")
print(tables)

['amend', 'avgreturns', 'company', 'form144', 'header', 'idfhist', 'idfnames', 'rule10b5', 's12', 's12names', 's12type1', 's12type2', 's12type3', 's12type4', 's12type5', 's12type6', 's12type7', 's12type8', 's34', 's34names', 's34type1', 's34type2', 's34type3', 's34type4', 's34type6', 'table1', 'table2']


### Download TNF Funding Holding Data - master file
- huge data, use 'for loop'

In [19]:
'''# Query to retrieve mutual fund holdings data
query_tnf_1 = """
    SELECT 
    fundno, fundname, 
    ticker, cusip, rdate, 
    assets/100 AS assets, 
    shares/1000000 AS shares_held, 
    shrout1
    
    FROM tfn.s12
    
    WHERE rdate BETWEEN '2020-01-01' AND '2023-12-31'
    AND country = 'UNITED STATES'
"""
#counts in millions(assets, shares, shrout1)
#tnf_1.info() check dataframe info'''

'# Query to retrieve mutual fund holdings data\nquery_tnf_1 = """\n    SELECT \n    fundno, fundname, \n    ticker, cusip, rdate, \n    assets/100 AS assets, \n    shares/1000000 AS shares_held, \n    shrout1\n    \n    FROM tfn.s12\n    \n    WHERE rdate BETWEEN \'2020-01-01\' AND \'2023-12-31\'\n    AND country = \'UNITED STATES\'\n"""\n#counts in millions(assets, shares, shrout1)\n#tnf_1.info() check dataframe info'

#### Testing -- finally works, LIKE'' use %% instead of % in the argument

#### Filter for index funds: End with IND, includes "INDEX, or ETF, or 500 or 1000 or 2000 or 3000"
- 500 for sp500, 1000 for Russell 1000, etc

In [73]:
simple_query = """
        SELECT  
            cusip, 
            fdate AS date,
            SUM(CASE
                WHEN fundname LIKE '%%IND' OR fundname LIKE '%%INDEX%%' OR fundname LIKE '%%ETF%%' OR 
                     fundname LIKE '%%500%%' OR fundname LIKE '%%1000%%' OR fundname LIKE '%%2000%%' OR fundname LIKE '%%3000%%'
                THEN shares/1000000/shrout1 
                ELSE -shares/1000000/shrout1
                END) AS ownership
    
        FROM tfn.s12

        WHERE 
                fdate BETWEEN '2023-09-01' AND '2023-12-31'
                AND country = 'UNITED STATES'
                AND shrout1 > 0
                
        GROUP BY 
            cusip, fdate
        ORDER BY 
            cusip, fdate
    
        LIMIT 20;
"""

try:
    simple_result = db.raw_sql(simple_query)
    print(simple_result)
except Exception as e:
    print("Error executing simple query:", e)


       cusip        date  ownership
0   00025510  2023-09-30  -0.047594
1   00025510  2023-12-31  -0.049696
2   00030410  2023-12-31  -0.000007
3   00032Q10  2023-09-30   0.013540
4   00032Q10  2023-12-31   0.023878
5   00036020  2023-09-30  -0.228980
6   00036020  2023-12-31  -0.238601
7   00036110  2023-09-30  -0.337879
8   00036110  2023-12-31  -0.332715
9   00037520  2023-09-30  -0.007078
10  00037520  2023-12-31  -0.008412
11  00037W10  2023-09-30  -0.007294
12  00037W10  2023-12-31  -0.007294
13  00038020  2023-09-30  -0.150957
14  00038020  2023-12-31  -0.091216
15  00039J20  2023-09-30  -0.025000
16  00039J60  2023-09-30  -0.047406
17  00039J60  2023-12-31  -0.029198
18  00064417  2023-12-31  -0.001323
19  00081T10  2023-09-30  -0.208811


#### Test - the accuracity of the filter

In [4]:
start_time_ins = time.time()

test_query_22q1 = """
    WITH Ownership AS (
        SELECT
            cusip, 
            fdate AS date,
            shares / 1000000 / shrout1 AS ownership,
            CASE
                WHEN fundname LIKE '%%IND' OR fundname LIKE '%%INDEX%%' OR fundname LIKE '%%ETF%%' OR 
                     fundname LIKE '%%500%%' OR fundname LIKE '%%1000%%' OR fundname LIKE '%%2000%%' OR fundname LIKE '%%3000%%'
                THEN 'Index Fund'
                ELSE 'Active Fund'
            END AS fund_type
        
        FROM 
            tfn.s12
        
        WHERE 
            fdate BETWEEN '2022-01-01' AND '2022-03-31'
            AND country = 'UNITED STATES'
            AND shrout1 > 0
    )
    SELECT 
        cusip,
        date,
        SUM(CASE WHEN fund_type = 'Index Fund' THEN ownership ELSE 0 END) AS index_own,
        SUM(CASE WHEN fund_type = 'Active Fund' THEN ownership ELSE 0 END) AS active_own
    FROM 
        Ownership
    GROUP BY 
        cusip, date
    ORDER BY 
        cusip, date
    
"""

try:
    test_result_22q1 = db.raw_sql(test_query_22q1)
    print(test_result_22q1)
except Exception as e:
    print("Error executing test query:", e)
    
end_time_ins = time.time()
execution_time_ins = end_time_ins - start_time_ins

print(f"Execution time for test ownership query: {execution_time_ins:.4f} seconds")

          cusip        date  index_own  active_own
0      00025510  2022-03-31   0.000000    0.045520
1      00032Q10  2022-03-31   0.044347    0.047593
2      00036020  2022-03-31   0.095057    0.348042
3      00036110  2022-03-31   0.118137    0.469007
4      00037520  2022-03-31   0.000019    0.010462
...         ...         ...        ...         ...
28182  Y9T91Q10  2022-03-31   0.000829    0.027172
28183  Y9T92M10  2022-03-31   0.000000    0.000121
28184  Y9T94Z10  2022-03-31   0.000000    0.000521
28185  Y9T95Z11  2022-03-31   0.000000    0.006446
28186  Y9T9HH10  2022-03-31   0.000000    0.009010

[28187 rows x 4 columns]
Execution time for test ownership query: 11.0150 seconds


In [6]:
check_cusip_1 = ['00025510','00036110','Y9T92M10']
df_check_1 = test_result_22q1[test_result_22q1['cusip'].isin(check_cusip_1)]

In [12]:
df_check_1

,cusip,date,index_own,active_own
0,00025510,2022-03-31,0.000000,0.045520
3,00036110,2022-03-31,0.118137,0.469007
28183,Y9T92M10,2022-03-31,0.000000,0.000121


#### the case grouping is correct sorting

#### check filter then - missing conditions
- fix a time, take few cusips to test
- take a cusip, test in a longer time horizon

In [5]:
query_check_1 = """
    SELECT 
    fundname, 
    cusip, 
    fdate AS date, 
    shares / 1000000 / shrout1 AS ownership
    
    FROM tfn.s12
    
    WHERE fdate BETWEEN '2022-01-01' AND '2022-03-31'
    AND country = 'UNITED STATES'
    AND shrout1 > 0
"""
try:
    check_result_1 = db.raw_sql(query_check_1)
    print(check_result_1)
except Exception as e:
    print("Error executing test query:", e)


                        fundname     cusip        date     ownership
0       FLORIDA STATE BOARD ADMI  00036020  2022-03-31  6.271509e-04
1       MORGAN STANLEY INTL FUND  00036020  2022-03-31  3.773585e-08
2       FLORIDA STATE BOARD ADMI  00036110  2022-03-31  4.390029e-03
3       FLORIDA STATE BOARD ADMI  00081T10  2022-03-31  6.552680e-04
4       BRJ BLACKROCK US SMALL C  00081T10  2022-03-31  6.563918e-05
...                          ...       ...         ...           ...
174927  DFA DIMENSIONAL EMERGING  Y9T95Z11  2022-03-31  6.917647e-05
174928  HARTFORD SCHRODERS DIVER  Y9T95Z11  2022-03-31  1.011765e-05
174929  TA WLD EX US CORE EQUITY  Y9T9HH10  2022-03-31  1.440667e-04
174930  DIMENSIONAL FDS ICVC EMG  Y9T9HH10  2022-03-31  2.093333e-04
174931  DFA DIMENSIONAL EMERGING  Y9T9HH10  2022-03-31  1.066667e-06

[2174932 rows x 4 columns]


In [11]:
df_check_2 = check_result_1[check_result_1['cusip'] == '00025510']
df_check_2

,fundname,cusip,date,ownership
381597,FIDELITY ADVR MULTI-ASSE,00025510,2022-03-31,0.024873
381598,ISHARES CDN S&P/TSX SMAL,00025510,2022-03-31,0.001300
381599,FIDELITY TACT HI CURRENC,00025510,2022-03-31,0.001946
381600,FIDELITY TACTICAL HI INC,00025510,2022-03-31,0.017400


In [14]:
own_sum_1 = df_check_2['ownership'].sum()
own_sum_1

0.04551973333333334

In [15]:
df_check_3 = check_result_1[check_result_1['cusip'] == 'Y9T92M10']
df_check_3

,fundname,cusip,date,ownership
329273,DFA EMER MRKT SML CAP SE,Y9T92M10,2022-03-31,1.209250e-04
329274,DFA EMER MRKT CORE EQUIT,Y9T92M10,2022-03-31,1.250000e-08


In [6]:
query_check_2 = """
    SELECT 
    fundname, 
    cusip, 
    fdate AS date, 
    shares / 1000000 / shrout1 AS ownership
    
    FROM tfn.s12
    
    WHERE fdate BETWEEN '2012-01-01' AND '2022-12-31'
    AND country = 'UNITED STATES'
    AND cusip = '00025510'
    AND shrout1 > 0
"""
try:
    check_result_2 = db.raw_sql(query_check_2)
    print(check_result_2)
except Exception as e:
    print("Error executing test query:", e)

                    fundname     cusip        date  ownership
0   PRESERVER ALTERNATIVE OP  00025510  2017-03-31   0.000417
1   PRESERVER ALTERNATIVE OP  00025510  2017-06-30   0.000417
2   ISHARES CDN S&P/TSX SMAL  00025510  2019-09-30   0.000950
3   ISHARES CDN S&P/TSX SMAL  00025510  2019-12-31   0.000923
4   ISHARES CDN S&P/TSX SMAL  00025510  2020-03-31   0.000966
..                       ...       ...         ...        ...
64  FIDELITY GBL MTHLY CURRE  00025510  2022-12-31   0.000043
65   FIDELITY CONS MNGD RISK  00025510  2022-12-31   0.000008
66  FIDELITY CONS INC PRIV P  00025510  2022-12-31   0.000026
67  FIDELITY TACTICAL HI INC  00025510  2022-12-31   0.009543
68  FIDELITY BALANCED INCOME  00025510  2022-12-31   0.000008

[69 rows x 4 columns]


In [7]:
unique_fundname_c2 = check_result_2['fundname'].unique()

# Display the unique fund names
print(unique_fundname_c2)

['PRESERVER ALTERNATIVE OP' 'ISHARES CDN S&P/TSX SMAL'
 'FIDELITY ADVR MULTI-ASSE' 'FIDELITY TACT HI CURRENC'
 'FIDELITY TACTICAL HI INC' 'FIDELITY CONVERTIBLE SEC'
 'FIDELITY U.S.MTHLY CURRE' 'FIDELITY GBL MTHLY CURRE'
 'ING INTL SMALLCAP MULTI-' 'FIDELITY GLOBAL MONTHLY'
 'FIDELITY CONSERVATIVE IN' 'FIDELITY CONS MNGD RISK'
 'FIDELITY CONS INC PRIV P' 'FIDELITY BALANCED INCOME' 'MD EQUITY FUND'
 'FIDELITY MONTHLY INC SER' 'FIDELITY BAL MNGD RISK P']


In [8]:
# Step 1: Create a new column to categorize 'Index' or 'Active' ownership
check_result_2['ownership_type'] = check_result_2['fundname'].apply(
    lambda x: 'Index' if 'S&P' in str(x) else 'Active'
)

# Step 2: Extract the year from the date column if not already done
check_result_2['year'] = pd.to_datetime(check_result_2['date']).dt.year

# Step 3: Group by year and ownership type, then sum the ownership for each group
ownership_summary = check_result_2.groupby(['year', 'ownership_type'])['ownership'].sum().reset_index()

# Step 4: Pivot the table to have separate columns for 'Index' and 'Active' ownership
ownership_summary_pivot = ownership_summary.pivot(index='year', columns='ownership_type', values='ownership').fillna(0)

# Rename columns for clarity
ownership_summary_pivot.columns = ['Active_Ownership', 'Index_Ownership']

# Display the final result
print(ownership_summary_pivot)


      Active_Ownership  Index_Ownership
year                                   
2017          0.000833         0.000000
2019          0.000000         0.001874
2020          0.009955         0.004786
2021          0.094916         0.005146
2022          0.193611         0.004774


In [3]:
tnf_22q1 = pd.read_csv('tnf_us_22q1.csv')

/tmp/ipykernel_3089/1583138986.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  tnf_22q1 = pd.read_csv('tnf_us_22q1.csv')


#### Check and Revise Index Filter - with each possible keywords

In [10]:
df_SP = tnf_22q1[tnf_22q1['fundname'].str.contains('S&P', case=False, na=False)].drop_duplicates(subset='fundname')
df_SP

,fundno,fundname,mgrcoab,country,shares,stkname,ticker,prc,shrout1
179491,81045,DIREXION MTHLY S&P500BEA,_EKGDK,UNITED STATES,2392275.0,GOLDMAN SACHS FOCUSED INTERN,FTIX,1.00,NaN
179492,81046,DIREXION MTHLY S&P500BUL,_EKGDK,UNITED STATES,11780000.0,FIDELITY COLCHESTER STREET T,FRGX,1.00,NaN
243567,1026,CARILLON S&P 500 INDEX,CARIL,UNITED STATES,7654.0,AFLAC INC,AFL,64.39,649.0
250997,1394,SCHWAB S&P 500 INDEX FUN,SCHWAB,UNITED STATES,1083443.0,AFLAC INC,AFL,64.39,649.0
309207,3169,NATIONWIDE S&P 500 INDEX,_EJZOE,UNITED STATES,20164.0,AFLAC INC,AFL,64.39,649.0
...,...,...,...,...,...,...,...,...,...
1489483,137073,ELKHORN S&P HI QUALITY P,NaN,UNITED STATES,253968.0,ALABAMA POWER CO,ALP,25.33,10.0
1493614,137373,DIREXION DAILY S&P500BUL,NaN,UNITED STATES,6128540.0,DREYFUS GOVERNMENT CASH MANA,DGCX,1.00,NaN
1504297,138479,SPDR S&P HEALTH CARE EQU,NaN,UNITED STATES,58154.0,ABBOTT LABORATORIES,ABT,118.36,1751.0
1532143,142230,PROSHARES S&P MIDCAP400D,NaN,UNITED STATES,173261.0,AMERICAN FINANCIAL GROUP INC,AFG,145.62,85.0


In [19]:
df_Ben = tnf_22q1[tnf_22q1['fundname'].str.contains('Bench', case=False, na=False)].drop_duplicates(subset='fundname')
df_Ben

,fundno,fundname,mgrcoab,country,shares,stkname,ticker,prc,shrout1
177896,80324,GMO BENCHMARK-FREE ALLOC,NaN,UNITED STATES,7187781.0,GMO U S EQUITY ALLOCATION FU,GMOD,25.37,NaN
1655086,152833,PACER BENCHMARK DATA INF,NaN,UNITED STATES,866581.0,AMERICAN TOWER CORP,AMT,251.22,456.0
1684927,156101,PACER BENCHMARK INDUSTRI,NaN,UNITED STATES,642180.0,AMERICOLD REALTY TRUST,COLD,27.88,269.0


In [17]:
df_NAS = tnf_22q1[tnf_22q1['fundname'].str.contains('Nasdaq', case=False, na=False)].drop_duplicates(subset='fundname')
df_NAS

,fundno,fundname,mgrcoab,country,shares,stkname,ticker,prc,shrout1
40028,29270,PROFUNDS ULTRA NASDAQ-10,_EYKCO,UNITED STATES,37708.0,ACTIVISION BLIZZARD INC,ATVI,80.11,781.0
52843,60100,PROFUNDS NASDAQ-100,_EYKCO,UNITED STATES,3622.0,ACTIVISION BLIZZARD INC,ATVI,80.11,781.0
179437,80944,DIREXION MTHLY NASDAQ100,_EKGDK,UNITED STATES,90080000.0,FIDELITY COLCHESTER STREET T,FRGX,1.00,NaN
182941,87874,NASDAQ PREMIUM INC & GR,NUVEEN,UNITED STATES,50618.0,AMC ENTERTAINMENT HOLDINGS I,AMC,24.64,517.0
210069,177464,DIREXION MONTHLY NASDAQ1,NaN,UNITED STATES,440000.0,FIDELITY COLCHESTER STREET T,FRGX,1.00,NaN
...,...,...,...,...,...,...,...,...,...
2039470,188104,INVESCO NASDAQ 100 ESG U,NaN,UNITED STATES,2486.0,ACTIVISION BLIZZARD INC,ATVI,80.11,781.0
2041617,188284,INVESCO ESG NASDAQ 100 E,NaN,UNITED STATES,711.0,ACTIVISION BLIZZARD INC,ATVI,80.11,781.0
2043480,188359,LINCOLN NASDAQ100 BUFFER,NaN,UNITED STATES,151638.0,LINCOLN SSGA NASDAQ100 INDEX,NaN,10.71,NaN
2049238,188770,STRATEGY SHARES NASDAQ 5,NaN,UNITED STATES,5990.0,ALERIAN MLP,AMLP,38.30,169.0


In [18]:
unique_fundname_NASDQ = df_NAS['fundname'].unique()

# Display the unique fund names
print(unique_fundname_NASDQ)

['PROFUNDS ULTRA NASDAQ-10' 'PROFUNDS NASDAQ-100'
 'DIREXION MTHLY NASDAQ100' 'NASDAQ PREMIUM INC & GR'
 'DIREXION MONTHLY NASDAQ1' 'HORIZONS NASDAQ-100COVER'
 'CALVERT VP NASDAQ 100 IN' 'RYDEX VARIABLE NASDAQ 10'
 'FIDELITY NASDAQ COMPOSIT' 'FIDELITY NASDAQ COMPOSTI'
 'RYDEX NASDAQ 100 FUND' 'CALIFORNIA INV NASDAQ 10'
 'RYDEX NASDAQ-100 2X STRA' 'OHIO NATL NASDAQ 100 IND'
 'VALIC I NASDAQ   100 IND' 'PROFUNDS VP NASDAQ100'
 'ISHARES NASDAQ BIOTECH I' 'USAA NASDAQ 100 INDEX FU'
 'FIRST NASDAQ100EQUAL WEI' 'FIRST NASDAQ100TECH SEC'
 'FIRST TR NASDAQ 100 EX T' 'FIRST NASDAQ CLEAN EDGE'
 'POWERSHARES NASDAQ 100 S' 'DIREXION NASDAQ-100EQUAL'
 'POWERSHARES NASDAQ INTER' 'FIRST TRUST NASDAQ RETAI'
 'FIRST TRUST NASDAQ TRAN' 'FIRST NASDAQ FOOD&BEVERA'
 'FIRST TR NASDAQ OIL & GA' 'PROSHARES ULTRA NASDAQ B'
 'FIRST TR NASDAQ PHARM ET' 'FIRST TRUST NASDAQ BANK'
 'FIRST NASDAQ CYBERSECURI' 'FIRST NASDAQ SEMICONDUCT'
 'SIREN NASDAQ NEXGEN ECON' 'FIRST TRUST NASDAQ AI AN'
 'STRATEGY SHARES NASDAQ 7

In [4]:
df_RUSS = tnf_22q1[tnf_22q1['fundname'].str.contains('RUSSELL', case=False, na=False)].drop_duplicates(subset='fundname')
df_RUSS

,fundno,fundname,mgrcoab,country,shares,stkname,ticker,prc,shrout1
179500,81083,PROSHARES ULTRA RUSSELL,_EYKCO,UNITED STATES,2848.0,AAON INC,AAON,55.73,53.0
189079,103944,RUSSELL AUSTRALIAN SELEC,NaN,UNITED STATES,4281293.0,WESTPAC BANKING CORP,WBCH,NaN,NaN
197573,140533,RUSSELL INVTS TAX-MNGD I,NaN,UNITED STATES,85406.0,ARC RESOURCES LTD,ARX/,13.39,690.0
240811,881,ING RUSSELL MID CAP INDE,PILCAP,UNITED STATES,22688.0,ADT INC,ADT,7.59,851.0
266684,1998,RUSSELL TAX MNGD U.S.LG,_EJXNZ,UNITED STATES,84596.0,AT&T INC,T,23.63,7143.0
...,...,...,...,...,...,...,...,...,...
1871258,168945,VANGUARD RUSSELL 2000 GR,NaN,UNITED STATES,30619.0,AAON INC,AAON,55.73,53.0
1887223,170214,INVESCO RUSSELL 1000 ENH,NaN,UNITED STATES,100.0,AFLAC INC,AFL,64.39,649.0
1888182,170333,VANGUARD RUSSELL 1000 IN,NaN,UNITED STATES,21250.0,ADT INC,ADT,7.59,851.0
1973272,177744,XTRACKERS RUSSELL 1000 U,NaN,UNITED STATES,210.0,AFLAC INC,AFL,64.39,649.0


In [5]:
unique_fundname_RUSSELL = df_RUSS['fundname'].unique()

# Display the unique fund names
print(unique_fundname_RUSSELL)

['PROSHARES ULTRA RUSSELL' 'RUSSELL AUSTRALIAN SELEC'
 'RUSSELL INVTS TAX-MNGD I' 'ING RUSSELL MID CAP INDE'
 'RUSSELL TAX MNGD U.S.LG' 'ISHARES RUSSELL 2000 IND'
 'CALVERT VP RUSSELL 2000' 'RYDEX RUSSELL 2000 1.5X'
 'RUSSELL STRATEGIC BOND F' 'RUSSELL INVESTMENTS STRA'
 'RUSSELL U.S. EQUITY POOL' 'RUSSELL EMER MRKT EQUITY'
 'RUSSELL OVERSEAS EQUITY' 'RUSSELL CANADIAN EQUITY'
 'METLIFE RUSSELL 2000 IND' 'RUSSELL THE U.S. SML CAP'
 'RUSSELL   THE CONT EURO' 'RUSSELL   THE U.K. EQUIT'
 'PROSHARES RUSSELL2000DIV' 'RUSSELL INS FDS REAL EST'
 'RUSSELL U.S. CORE EQUITY' 'RUSSELL U.S.SML CAP&MID'
 'RUSSELL GBL REAL ESTATE' 'RUSSELL U.S.QUANTITATIVE'
 'RUSSELL EMER MRKT FUND' 'RUSSELL TAX MNGD MID&SML'
 'ISHARES RUSSELL2000 GR I' 'ISHARES RUSSELL1000VAL I'
 'ISHARES RUSSELL 3000 IND' 'ISHARES RUSSELL2000VAL I'
 'ISHARES RUSSELL 1000 IND' 'ISHARES RUSSELL1000 GR I'
 'ISHARES RUSSELL3000 GR I' 'ISHARES RUSSELL3000VAL I'
 'ISHARES RUSSELL MIDCAP I' 'ISHARES RUSSELL MIDCAP G'
 'ISHARES RUSSELL MID

In [11]:
df_DOW = tnf_22q1[tnf_22q1['fundname'].str.contains('DOW |DOW30', case=False, na=False)].drop_duplicates(subset='fundname')
df_DOW

,fundno,fundname,mgrcoab,country,shares,stkname,ticker,prc,shrout1
39779,20450,PROFUNDS ULTRA DOW 30,_EYKCO,UNITED STATES,5297.0,AMERICAN EXPRESS CO,AXP,187.00,757.0
156218,190601,INVESCO DOW JONES ISLAMI,NaN,UNITED STATES,151.0,AGCO CORP,AGCO,146.03,75.0
330270,5705,ISHARES DOW JONES US IND,_EJZOE,UNITED STATES,40560.0,ADT INC,ADT,7.59,851.0
469763,37696,ISHARES DOW JONES TRAN A,_EJZOE,UNITED STATES,135589.0,AIR TRANSPORT SERVICES GROUP,ATSG,33.45,74.0
470252,38387,ISHARES DOW JONES SEL DI,_EJZOE,UNITED STATES,14662051.0,AT&T INC,T,23.63,7143.0
510334,43945,RYDEX DOW JONES INDUSTRI,NaN,UNITED STATES,3185.0,AMERICAN EXPRESS CO,AXP,187.00,757.0
578945,51378,SPDR DOW JONES INDUSTRIA,STSTBT,UNITED STATES,5834655.0,AMERICAN EXPRESS CO,AXP,187.00,757.0
617769,56302,WELLS FARGO ADV DOW JONE,_IRSLH,UNITED STATES,34.0,AAON INC,AAON,55.73,53.0
643911,57032,ISHARES DOW JONES US ENG,_EJZOE,UNITED STATES,817271.0,ANTERO MIDSTREAM CORP,AM,10.87,478.0
643949,57033,ISHARES DOW JONES US HEA,_EJZOE,UNITED STATES,953056.0,ABBOTT LABORATORIES,ABT,118.36,1751.0


In [12]:
unique_fundname_DOW = df_DOW['fundname'].unique()

# Display the unique fund names
print(unique_fundname_DOW)

['PROFUNDS ULTRA DOW 30' 'INVESCO DOW JONES ISLAMI'
 'ISHARES DOW JONES US IND' 'ISHARES DOW JONES TRAN A'
 'ISHARES DOW JONES SEL DI' 'RYDEX DOW JONES INDUSTRI'
 'SPDR DOW JONES INDUSTRIA' 'WELLS FARGO ADV DOW JONE'
 'ISHARES DOW JONES US ENG' 'ISHARES DOW JONES US HEA'
 'ISHARES DOW JONES US CON' 'ISHARES DOW JONES US FIN'
 'ISHARES DOW JONES US TEC' 'ISHARES DOW JONES US BAS'
 'ISHARES DOW JONES US TEL' 'ISHARES DOW JONES US UTI'
 'ISHARES DOW JONES US REA' 'SPDR DOW JONES SML CAP G'
 'SPDR DOW JONES SML CAP V' 'SPDR DOW JONES U.S.LG CA'
 'SPDR DOW JONES LG CAP VA' 'SPDR DOW JONES TOT MARKE'
 'SPDR DOW JONES REIT ETF' 'SPDR DOW JONES MID CAP G'
 'SPDR DOW JONES MID CAP V' 'SPDR DOW JONES LARGE CAP'
 'SPDR DOW JONES MID CAP E' 'SPDR DOW JONES SMALL CAP'
 'ISHARES DOW JONES U.S.BR' 'ISHARES DOW JONES U.S.HO'
 'ISHARES DOW JONES U.S.IN' 'ISHARES DOW JONES U.S.OI'
 'ISHARES DOW JONES U.S.AE' 'ISHARES DOW JONES U.S.HE'
 'ISHARES DOW JONES U.S.ME' 'ISHARES DOW JONES U.S.PH'
 'ISHARES DOW 

In [15]:
df_FTSE = tnf_22q1[tnf_22q1['fundname'].str.contains('FTSE', case=False, na=False)].drop_duplicates(subset='fundname')
df_FTSE

,fundno,fundname,mgrcoab,country,shares,stkname,ticker,prc,shrout1
188868,99568,FRANKLIN FTSE RUSSIA ETF,NaN,UNITED STATES,174496.0,AEROFLOTROSSIYSKIYE AVIALINI,AFLT,0.48,2445.0
321355,4763,VANGUARD FTSE 250 UCITS,NaN,UNITED STATES,424425.0,VITAFORT INTL CORP,VRFT,0.00,21.0
387721,15406,VANGUARD FTSE CDN CAPPED,NaN,UNITED STATES,573881.0,ALLIED PROPERTIES REAL ESTAT,APU,37.31,128.0
527974,44903,ISHARES FTSE/XINHUA CHIN,_EJZOE,UNITED STATES,8610000.0,BLACKROCK CASH FUNDS TREASUR,NaN,1.00,NaN
533647,45851,ISHARES FTSE KLD SEL SOC,_EJZOE,UNITED STATES,128284.0,ADOBE INC,ADBE,455.62,473.0
...,...,...,...,...,...,...,...,...,...
1841613,166982,DIREXION DAILY FTSE EURO,NaN,UNITED STATES,12749870.0,DREYFUS GOVERNMENT CASH MANA,DGCX,1.00,NaN
1864858,168442,GLOBAL X FTSE SOUTHEAST,NaN,UNITED STATES,113768.0,ADVANCED INFO SERVICE PCL,ADVA,7.00,2974.0
1899557,171000,VANGUARD FTSE DVLP EX NT,NaN,UNITED STATES,1400.0,BROTHER INDS LTD,BRTH,18.39,262.0
1945652,175171,FRANKLIN FTSE ASIA EX JA,NaN,UNITED STATES,800.0,SHANDONG PUBLISHING MEDIA CO,6010,1.02,2087.0


In [13]:
df_MSCI= tnf_22q1[tnf_22q1['fundname'].str.contains('MSCI', case=False, na=False)].drop_duplicates(subset='fundname')
df_MSCI

,fundno,fundname,mgrcoab,country,shares,stkname,ticker,prc,shrout1
151872,187836,INVESCO MSCI EUROPE ESG,NaN,UNITED STATES,529.0,VOLVO AKTIEBOLAGET,VOLV,18.80,1588.0
152147,187962,INVESCO MSCI USA ESG CLI,NaN,UNITED STATES,240.0,AFLAC INC,AFL,64.39,649.0
164066,4835,DEUTSCHE X-TRACKERS MSCI,NaN,UNITED STATES,303.0,DEUTSCHE DWS MONEY MARKET TR,ICAX,1.00,NaN
246106,1173,ISHARES MSCI ISRAEL CAPP,_EJZOE,UNITED STATES,70000.0,BLACKROCK CASH FUNDS TREASUR,NaN,1.00,NaN
268748,2074,ISHARES MSCI CNDA INDEX,_EJZOE,UNITED STATES,1098407.0,AGNICO EAGLE MINES LTD,AEM/,61.20,456.0
...,...,...,...,...,...,...,...,...,...
2039711,188120,GLOBAL X MSCI VIETNAM ET,NaN,UNITED STATES,27475.0,AN PHAT HOLDINGS JSC,APH,1.22,244.0
2040700,188219,INVESCO MSCI JAPAN ESG C,NaN,UNITED STATES,188.0,ADVANTEST CORP,6857,79.48,200.0
2046130,188488,IMPACT SHARES MSCI GLOBA,NaN,UNITED STATES,104.0,ADOBE INC,ADBE,455.62,473.0
2048144,188741,INVESCO MSCI WORLD ESG C,NaN,UNITED STATES,87.0,AFLAC INC,AFL,64.39,649.0


In [16]:
df_TSX= tnf_22q1[tnf_22q1['fundname'].str.contains('TSX', case=False, na=False)].drop_duplicates(subset='fundname')
df_TSX

,fundno,fundname,mgrcoab,country,shares,stkname,ticker,prc,shrout1
338186,6357,ISHARES S&P/TSX CAPPED F,_EJZOE,UNITED STATES,1008988.0,BANK OF MONTREAL,BMO/,117.70,648.0
667527,60269,ISHARES S&P/TSX 60 INDEX,_EJZOE,UNITED STATES,2332367.0,AGNICO EAGLE MINES LTD,AEM/,61.20,456.0
708321,63566,ISHARES S&P/TSX CAPPED I,_EJZOE,UNITED STATES,163368.0,ABSOLUTE SOFTWARE CORP,ABT/,8.44,51.0
708347,63567,ISHARES S&P/TSX CAPPED C,_EJZOE,UNITED STATES,250197.0,ATS AUTOMATION TOOLING SYSTE,ATA/,36.08,92.0
708586,63568,ISHARES S&P/TSX COMPLETI,_EJZOE,UNITED STATES,22732.0,ATS AUTOMATION TOOLING SYSTE,ATA/,36.08,92.0
708765,63571,ISHARES S&P/TSX GBL GOLD,_EJZOE,UNITED STATES,1710154.0,AGNICO EAGLE MINES LTD,AEM/,61.20,456.0
840178,76824,ISHARES S&P/TSX CAPPED R,_EJZOE,UNITED STATES,2345584.0,ALLIED PROPERTIES REAL ESTAT,APU,37.31,128.0
1016078,84457,ISHARES S&P/TSX CAPPED M,_EJZOE,UNITED STATES,178449.0,AGNICO EAGLE MINES LTD,AEM/,61.20,456.0
1034532,88197,ISHARES CDN S&P/TSX SMAL,_EJZOE,UNITED STATES,19500.0,A AND W REVENUE ROYALTIES IN,AWU,34.61,15.0
1345309,117531,POWERSHARES S&P/TSX COMP,NaN,UNITED STATES,280929.0,ALGONQUIN POWER & UTILITIES,AQN/,15.51,674.0


In [18]:
tnf_22q1['ownership'] = tnf_22q1['shares'] /1000000/ tnf_22q1['shrout1']

In [78]:
tnf_22q1

,fundno,fundname,mgrcoab,country,shares,stkname,ticker,prc,shrout1,ownership
0,73,BMO NORTH AMERICAN DIV F,_EJYTQ,UNITED STATES,31324.0,AGCO CORP,AGCO,146.03,75.0,0.000418
1,73,BMO NORTH AMERICAN DIV F,_EJYTQ,UNITED STATES,157048.0,AES CORP,AES,25.73,667.0,0.000235
2,73,BMO NORTH AMERICAN DIV F,_EJYTQ,UNITED STATES,11500.0,ABBOTT LABORATORIES,ABT,118.36,1751.0,0.000007
3,73,BMO NORTH AMERICAN DIV F,_EJYTQ,UNITED STATES,7162.0,ACUITY BRANDS INC,AYI,189.30,35.0,0.000205
4,73,BMO NORTH AMERICAN DIV F,_EJYTQ,UNITED STATES,3900.0,AIR PRODUCTS AND CHEMICALS I,APD,249.91,222.0,0.000018
...,...,...,...,...,...,...,...,...,...,...
2071576,191191,PROSHARES METAVERSE ETF,NaN,UNITED STATES,202.0,WILLIAMS-SONOMA INC,WSM,145.00,69.0,0.000003
2071577,191191,PROSHARES METAVERSE ETF,NaN,UNITED STATES,89768.0,WIMI HOLOGRAM CLOUD INC,WIMI,2.67,77.0,0.001166
2071578,191191,PROSHARES METAVERSE ETF,NaN,UNITED STATES,10061.0,ZYNGA INC,ZNGA,9.24,1137.0,0.000009
2071579,191191,PROSHARES METAVERSE ETF,NaN,UNITED STATES,911.0,AMBARELLA INC,NaN,104.92,38.0,0.000024


In [19]:
index_keywords = ['INDEX', 'ETF', 'S&P', 'NASDAQ', 'MSCI', 'FTSE', 'TSX', 'PASSIVE', '500', '2000']
tnf_22q1['group'] = tnf_22q1['fundname'].apply(
    lambda x: 'Index' if (any(keyword in str(x) for keyword in index_keywords) 
                          and not str(x).startswith('NASDAQ'))
                          or ('Russell' in str(x) and not str(x).startswith('Russell')) 
                          or 'Russell 2000' in str(x)
                          or 'DOW ' in str(x)  # Contains 'DOW ' (with space)
                          or 'DOW30' in str(x)
                          or str(x).endswith('IND') else 'Active'
)

# Step 2: Sum shares by cusip and group
df_grouped_22q1 = tnf_22q1.groupby(['ticker', 'group'])['ownership'].sum().reset_index()

In [20]:
df_grouped_22q1

,ticker,group,ownership
0,0000,Active,11.305801
1,0000,Index,0.052946
2,0001,Active,0.106785
3,0001,Index,0.050181
4,0002,Active,0.159685
...,...,...,...
35738,ZYXI,Active,0.055323
35739,ZYXI,Index,0.103581
35740,ZZZ,Active,0.061697
35741,ZZZ,Index,0.008247


In [21]:
# Filter the DataFrame for ownership smaller than 0.0001
df_small = df_grouped_22q1[df_grouped_22q1['ownership'] < 0.0001]

# Display the filtered rows
print(df_small)

      ticker   group     ownership
201     0121  Active  4.998765e-05
202     0121   Index  5.083951e-05
259     0179   Index  1.863636e-06
346     0299   Index  1.020000e-06
352     0302   Index  6.850192e-05
...      ...     ...           ...
35651   ZKIN  Active  1.379310e-07
35660   ZMEN   Index  7.411765e-05
35682    ZQK  Active  1.686047e-07
35712    ZUR  Active  1.410769e-05
35716   ZURV   Index  1.665543e-05

[3778 rows x 3 columns]


In [ ]:
#### Testing - faster way

### Combined Active and Index Ownership

In [19]:
start_time_ins = time.time()

test_query_1 = """
    WITH Ownership AS (
        SELECT
            cusip, 
            fdate AS date,
            shares / 1000000 / shrout1 AS ownership,
            CASE
                WHEN fundname LIKE '%%IND' OR fundname LIKE '%%INDEX%%' OR fundname LIKE '%%ETF%%' OR 
                     fundname LIKE '%%500%%' OR fundname LIKE '%%1000%%' OR fundname LIKE '%%2000%%' OR fundname LIKE '%%3000%%'
                THEN 'Index Fund'
                ELSE 'Active Fund'
            END AS fund_type
        
        FROM 
            tfn.s12
        
        WHERE 
            fdate BETWEEN '2023-09-01' AND '2023-12-31'
            AND country = 'UNITED STATES'
            AND shrout1 > 0
    )
    SELECT 
        cusip,
        date,
        SUM(CASE WHEN fund_type = 'Index Fund' THEN ownership ELSE 0 END) AS index_own,
        SUM(CASE WHEN fund_type = 'Active Fund' THEN ownership ELSE 0 END) AS active_own
    FROM 
        Ownership
    GROUP BY 
        cusip, date
    ORDER BY 
        cusip, date
    
    LIMIT 100;
"""

try:
    test_result_1 = db.raw_sql(test_query_1)
    print(test_result_1)
except Exception as e:
    print("Error executing test query:", e)
    
end_time_ins = time.time()
execution_time_ins = end_time_ins - start_time_ins

print(f"Execution time for test ownership query: {execution_time_ins:.4f} seconds")

       cusip        date  index_own  active_own
0   00025510  2023-09-30   0.000000    0.047594
1   00025510  2023-12-31   0.000000    0.049696
2   00030410  2023-12-31   0.000000    0.000007
3   00032Q10  2023-09-30   0.051871    0.038331
4   00032Q10  2023-12-31   0.056873    0.032996
..       ...         ...        ...         ...
95  00162Q85  2023-09-30   0.000000    0.000225
96  00162Q85  2023-12-31   0.000000    0.000156
97  00164V10  2023-09-30   0.121951    0.323123
98  00164V10  2023-12-31   0.125908    0.420986
99  00165C20  2023-09-30   0.000391    0.000130

[100 rows x 4 columns]
Execution time for institutional ownership query: 0.2430 seconds


In [24]:
file_name = "tnf_own_combined.csv"

start_year = 1993
current_year = datetime.now().year

for year in range(start_year, current_year + 1):
    
    print(f"Downloading data for the year {year}...")
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    
    query = f"""
    WITH Ownership AS (
        SELECT
            cusip, 
            fdate AS date,
            shares / 1000000 / shrout1 AS ownership,
            CASE
                WHEN fundname LIKE '%%IND' OR fundname LIKE '%%INDEX%%' OR fundname LIKE '%%ETF%%' OR 
                     fundname LIKE '%%500%%' OR fundname LIKE '%%1000%%' OR fundname LIKE '%%2000%%' OR fundname LIKE '%%3000%%'
                THEN 'Index Fund'
                ELSE 'Active Fund'
            END AS fund_type
        
        FROM 
            tfn.s12
        
        WHERE 
            fdate BETWEEN '{start_date}' AND '{end_date}'
            AND country = 'UNITED STATES'
            AND shrout1 > 0
        )
        SELECT 
            cusip,
            date,
            SUM(CASE WHEN fund_type = 'Index Fund' THEN ownership ELSE 0 END) AS index_own,
            SUM(CASE WHEN fund_type = 'Active Fund' THEN ownership ELSE 0 END) AS active_own
        FROM 
            Ownership
        GROUP BY 
            cusip, date
        ORDER BY 
            cusip, date;
    """
    
    # Fetch data for the current year
    year_data = db.raw_sql(query)
    
    # Append the data to the CSV file
    if year == start_year:
        # Write with header if it's the first year
        year_data.to_csv(file_name, mode='w', header=True, index=False)
    else:
        # Append without header if it's not the first year
        year_data.to_csv(file_name, mode='a', header=False, index=False)
    
    print(f"Data for year {year} appended to '{file_name}'")

Data for year 1993 appended to 'tnf_own_combined.csv'
Data for year 1994 appended to 'tnf_own_combined.csv'
Data for year 1995 appended to 'tnf_own_combined.csv'
Data for year 1996 appended to 'tnf_own_combined.csv'
Data for year 1997 appended to 'tnf_own_combined.csv'
Data for year 1998 appended to 'tnf_own_combined.csv'
Data for year 1999 appended to 'tnf_own_combined.csv'
Data for year 2000 appended to 'tnf_own_combined.csv'
Data for year 2001 appended to 'tnf_own_combined.csv'
Data for year 2002 appended to 'tnf_own_combined.csv'
Data for year 2003 appended to 'tnf_own_combined.csv'
Data for year 2004 appended to 'tnf_own_combined.csv'
Data for year 2005 appended to 'tnf_own_combined.csv'
Data for year 2006 appended to 'tnf_own_combined.csv'
Data for year 2007 appended to 'tnf_own_combined.csv'
Data for year 2008 appended to 'tnf_own_combined.csv'
Data for year 2009 appended to 'tnf_own_combined.csv'
Data for year 2010 appended to 'tnf_own_combined.csv'
Data for year 2011 appended 

In [25]:
# check
check_own_1 = pd.read_csv('tnf_own_combined.csv')

### Institutional Ownership

In [50]:
df_ins = pd.read_csv("tnf_data_ins.csv")#, nrows=10000000)
#df_ins[df_ins.date == "1999-03-31"]
len(df_ins.cusip.unique())

45390

In [51]:
df_ins

,ticker,cusip,date,ins_own
0,NaN,00786910,1999-03-31,0.002258
1,NaN,07734710,1999-03-31,0.004380
2,NaN,15942020,1999-03-31,0.000853
3,NaN,17118810,1999-03-31,0.000750
4,NaN,21671220,1999-03-31,0.005680
...,...,...,...,...
48079766,ZYXI,98986M10,2024-06-30,0.001200
48079767,ZYXI,98986M10,2024-06-30,0.000150
48079768,ZYXI,98986M10,2024-06-30,0.000599
48079769,ZYXI,98986M10,2024-06-30,0.036515


In [49]:
#len(check_own_1.cusip.unique())*4*(24+8)
check_own_1[check_own_1.date == "2023-03-31"]
len(check_own_1.cusip.unique())

63240

### Get institutional holding info(s34)
- calculate institutional ownership

In [17]:
'''start_time_ins = time.time()

query_ins_1 = """
    SELECT 
    ticker, cusip, fdate AS date,  
    shares/1000000/shrout1 AS ins_own
    
    FROM tfn.s34
    
    WHERE fdate BETWEEN '2023-09-01' AND '2023-12-31'
    AND country = 'UNITED STATES'
    AND shrout1 > 0
    AND shares/1000000/shrout1 >= 0.0001
"""

ins_1 = db.raw_sql(query_ins_1)

end_time_ins = time.time()
execution_time_ins = end_time_ins - start_time_ins

# Print the number of records and execution time for the institutional ownership query
print(f"Number of records returned: {len(ins_1)}")
print(f"Execution time for institutional ownership query: {execution_time_ins:.4f} seconds")

#with constraint(AND shares/1000000/shrout1 >= 0.0001) saves half of the time'''

'start_time_ins = time.time()\n\nquery_ins_1 = """\n    SELECT \n    ticker, cusip, fdate AS date,  \n    shares/1000000/shrout1 AS ins_own\n    \n    FROM tfn.s34\n    \n    WHERE fdate BETWEEN \'2023-09-01\' AND \'2023-12-31\'\n    AND country = \'UNITED STATES\'\n    AND shrout1 > 0\n    AND shares/1000000/shrout1 >= 0.0001\n"""\n\nins_1 = db.raw_sql(query_ins_1)\n\nend_time_ins = time.time()\nexecution_time_ins = end_time_ins - start_time_ins\n\n# Print the number of records and execution time for the institutional ownership query\nprint(f"Number of records returned: {len(ins_1)}")\nprint(f"Execution time for institutional ownership query: {execution_time_ins:.4f} seconds")\n\n#with constraint(AND shares/1000000/shrout1 >= 0.0001) saves half of the time'

In [12]:
ins_1

,ticker,cusip,date,ins_own
0,None,00141G87,2023-09-30,0.000668
1,None,00175J10,2023-09-30,0.000382
2,None,00175J10,2023-09-30,0.028898
3,None,00175J10,2023-09-30,0.000201
4,None,00175J10,2023-09-30,0.000407
...,...,...,...,...
467647,ZYXI,98986M10,2023-12-31,0.011533
467648,ZYXI,98986M10,2023-12-31,0.001409
467649,ZYXI,98986M10,2023-12-31,0.034735
467650,ZYXI,98986M10,2023-12-31,0.001144


#### check data and test whether group by works well
- multiple entries of shares under same cusip and date are holdings by different managers(mgrno)

In [61]:
query_ins_check_1 = """
    SELECT 
    cusip, fdate AS date,  
    sum(shares/1000000/shrout1) AS ins_own
    
    FROM tfn.s34
    
    WHERE fdate BETWEEN '2023-09-01' AND '2023-12-31'
    AND country = 'UNITED STATES'
    AND shrout1 > 0
    AND shares/1000000/shrout1 <= 1
    AND shares/1000000/shrout1 >= 0.000001

    GROUP BY 
        cusip, fdate
    ORDER BY 
        cusip, fdate;
"""

check_ins_1 = db.raw_sql(query_ins_check_1)
#make ownership as percentage (shrout1 in million, but time 100 for percentage)

In [28]:
query_ins_check_2 = """
    SELECT 
    cusip, fdate AS date,  
    mgrno,
    shares,
    shrout1,
    shares/1000000/shrout1 AS ins_own
    
    FROM tfn.s34
    
    WHERE fdate BETWEEN '2023-12-01' AND '2023-12-31'
    AND country = 'UNITED STATES'
    AND shrout1 > 0
    AND cusip = '98986M10'
    
"""

check_ins_2 = db.raw_sql(query_ins_check_2)

In [30]:
total_ownership = check_ins_2['ins_own'].fillna(0).sum()
total_ownership

0.25623767647058826

In [62]:
check_ins_1

,cusip,date,ins_own
0,00030410,2023-09-30,0.000018
1,00030410,2023-12-31,0.000018
2,00032Q10,2023-09-30,0.554345
3,00032Q10,2023-12-31,0.509033
4,00036020,2023-09-30,0.671618
...,...,...,...
24027,Y9728A10,2023-12-31,0.000087
24028,Y9863Z12,2023-09-30,0.000007
24029,Y9899X10,2023-09-30,0.000325
24030,Y9899X10,2023-12-31,0.000321


In [33]:
df_98986M10 = check_ins_1[check_ins_1['cusip'] == '98986M10']

In [34]:
df_98986M10

,cusip,date,ins_own
22385,98986M10,2023-09-30,25.986808
22386,98986M10,2023-12-31,25.623768


#### grasp data from wrds

In [63]:
file_name = "tnf_data_ins.csv"

start_year = 1993
current_year = datetime.now().year

for year in range(start_year, current_year + 1):
    
    print(f"Downloading data for the year {year}...")
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    
    query = f"""
        SELECT  
        cusip, fdate AS date,  
        sum(shares/1000000/shrout1) AS ins_own
        
        FROM tfn.s34
        
        WHERE fdate BETWEEN '{start_date}' AND '{end_date}'
        AND country = 'UNITED STATES'
        AND shrout1 > 0
        AND shares/1000000/shrout1 <= 1
        AND shares/1000000/shrout1 >= 0.00001

        GROUP BY 
            cusip, fdate
        ORDER BY 
            cusip, fdate;
    """
    
    # Fetch data for the current year
    year_data = db.raw_sql(query)
    
    # Append the data to the CSV file
    if year == start_year:
        # Write with header if it's the first year
        year_data.to_csv(file_name, mode='w', header=True, index=False)
    else:
        # Append without header if it's not the first year
        year_data.to_csv(file_name, mode='a', header=False, index=False)
    
    print(f"Data for year {year} appended to '{file_name}'")

Data for year 1993 appended to 'tnf_data_ins.csv'
Data for year 1994 appended to 'tnf_data_ins.csv'
Data for year 1995 appended to 'tnf_data_ins.csv'
Data for year 1996 appended to 'tnf_data_ins.csv'
Data for year 1997 appended to 'tnf_data_ins.csv'
Data for year 1998 appended to 'tnf_data_ins.csv'
Data for year 1999 appended to 'tnf_data_ins.csv'
Data for year 2000 appended to 'tnf_data_ins.csv'
Data for year 2001 appended to 'tnf_data_ins.csv'
Data for year 2002 appended to 'tnf_data_ins.csv'
Data for year 2003 appended to 'tnf_data_ins.csv'
Data for year 2004 appended to 'tnf_data_ins.csv'
Data for year 2005 appended to 'tnf_data_ins.csv'
Data for year 2006 appended to 'tnf_data_ins.csv'
Data for year 2007 appended to 'tnf_data_ins.csv'
Data for year 2008 appended to 'tnf_data_ins.csv'
Data for year 2009 appended to 'tnf_data_ins.csv'
Data for year 2010 appended to 'tnf_data_ins.csv'
Data for year 2011 appended to 'tnf_data_ins.csv'
Data for year 2012 appended to 'tnf_data_ins.csv'


### Load Data and Process

#### Summariza Data and Check

In [4]:
df_own= pd.read_csv('tnf_own_combined.csv')
df_ins= pd.read_csv('tnf_data_ins.csv')

In [64]:
df_ins= pd.read_csv('tnf_data_ins.csv')

In [67]:
def count_non_zero(cusip_series, ownership_series):
   return len(cusip_series[ownership_series != 0].unique())

In [5]:
df_own['date'] = pd.to_datetime(df_own['date'])

# Step 3: Extract year
df_own['year'] = df_own['date'].dt.year

# Group by year and calculate summary statistics
summary_own = df_own.groupby('year').agg(
    average_index_ownership=('index_own', 'mean'),
    median_index_ownership=('index_own', 'median'),
    average_active_ownership=('active_own', 'mean'),
    median_active_ownership=('active_own', 'median'),
     index_stock_count=('cusip', lambda x: count_non_zero(x, df_own.loc[x.index, 'index_own'])),
    active_stock_count=('cusip', lambda x: count_non_zero(x, df_own.loc[x.index, 'active_own']))  # Count unique stocks
).reset_index()

# Display the summarized DataFrame
print(summary_own)

    year  average_index_ownership  median_index_ownership  \
0   1999                 0.003296                0.000012   
1   2000                 0.003347                0.000030   
2   2001                 0.004568                0.000091   
3   2002                 0.005910                0.000163   
4   2003                 0.007174                0.000552   
5   2004                 0.011373                0.001587   
6   2005                 0.011009                0.002049   
7   2006                 0.009480                0.001136   
8   2007                 0.010258                0.000400   
9   2008                 0.012037                0.000342   
10  2009                 0.015442                0.000330   
11  2010                 0.017978                0.000368   
12  2011                 0.018154                0.000098   
13  2012                 0.020081                0.000421   
14  2013                 0.020175                0.000692   
15  2014                

In [68]:
df_ins['date'] = pd.to_datetime(df_ins['date'])

# Extract year
df_ins['year'] = df_ins['date'].dt.year

# Step 4: Group by year and calculate summary statistics
summary_inst= df_ins.groupby('year').agg(
    average_instit_ownership=('ins_own', 'mean'),
    median_instit_ownership=('ins_own', 'median'),
    min_instit_ownership=('ins_own', 'min'),
    max_instit_ownership=('ins_own', 'max'),
    q25_instit_ownership=('ins_own', lambda x: x.quantile(0.25)),
    q75_instit_ownership=('ins_own', lambda x: x.quantile(0.75)),
    skewness_instit_ownership=('ins_own', lambda x: skew(x, nan_policy='omit')),
    kurtosis_instit_ownership=('ins_own', lambda x: kurtosis(x, nan_policy='omit')),
    instit_stock_count=('cusip', lambda x: count_non_zero(x, df_ins.loc[x.index, 'ins_own']))  # Count unique stocks
).reset_index()

# Display the summarized DataFrame
print(summary_inst)

    year  average_instit_ownership  median_instit_ownership  \
0   1999                  0.238187                 0.142656   
1   2000                  0.221897                 0.117480   
2   2001                  0.236479                 0.119187   
3   2002                  0.249787                 0.118629   
4   2003                  0.270349                 0.138881   
5   2004                  0.287599                 0.148776   
6   2005                  0.304048                 0.167363   
7   2006                  0.303505                 0.165042   
8   2007                  0.319727                 0.182922   
9   2008                  0.315660                 0.191456   
10  2009                  0.294768                 0.181383   
11  2010                  0.319254                 0.196134   
12  2011                  0.296731                 0.164777   
13  2012                  0.308294                 0.191818   
14  2013                  0.311460                 0.19

In [69]:
oultier_inst= df_ins.groupby('year').agg(
    q90_instit_ownership=('ins_own', lambda x: x.quantile(0.90)),
    q95_instit_ownership=('ins_own', lambda x: x.quantile(0.95)),
    q98_instit_ownership=('ins_own', lambda x: x.quantile(0.98)),
    q99_instit_ownership=('ins_own', lambda x: x.quantile(0.99)),
    instit_stock_count=('cusip', lambda x: count_non_zero(x, df_ins.loc[x.index, 'ins_own']))  # Count unique stocks
).reset_index()

# Display the summarized DataFrame
print(oultier_inst)

    year  q90_instit_ownership  q95_instit_ownership  q98_instit_ownership  \
0   1999              0.645532              0.743001              0.831897   
1   2000              0.627528              0.730363              0.818056   
2   2001              0.675454              0.779754              0.865709   
3   2002              0.708507              0.800408              0.887207   
4   2003              0.740826              0.831748              0.913791   
5   2004              0.779507              0.864790              0.944435   
6   2005              0.803223              0.885808              0.971816   
7   2006              0.799578              0.892599              0.982424   
8   2007              0.837588              0.932570              1.035046   
9   2008              0.818386              0.914344              1.013378   
10  2009              0.767055              0.843756              0.918129   
11  2010              0.826797              0.904137            

In [16]:
ins_2024 = df_ins[df_ins['year'] == 2024]

In [17]:
ins_2024

,cusip,date,ins_own,year
885482,00032Q10,2024-03-31,0.527503,2024
885483,00032Q10,2024-06-30,0.455760,2024
885484,00036020,2024-03-31,0.652461,2024
885485,00036020,2024-06-30,0.676160,2024
885486,00036110,2024-03-31,0.873030,2024
...,...,...,...,...
906570,Y9724910,2024-06-30,0.000998,2024
906571,Y9728A10,2024-03-31,0.000103,2024
906572,Y9728A10,2024-06-30,0.000119,2024
906573,Y9899X10,2024-03-31,0.000317,2024


In [18]:
max_ins_2024 = ins_2024.nlargest(20, 'ins_own')
max_ins_2024

,cusip,date,ins_own,year
899413,74738J30,2024-03-31,10493.231653,2024
903924,98873N20,2024-03-31,21.584665,2024
900545,81621230,2024-03-31,8.527154,2024
887481,08249020,2024-06-30,7.647670,2024
889506,23257B20,2024-06-30,6.769378,2024
888807,17175720,2024-03-31,4.250320,2024
895770,49446R68,2024-06-30,4.202732,2024
890664,29287L70,2024-06-30,4.049014,2024
900135,78464A71,2024-03-31,3.691165,2024
896323,53700T74,2024-06-30,3.340967,2024


In [19]:
ins_outlier = df_ins[df_ins['cusip'] == '74738J30']
ins_outlier

,cusip,date,ins_own,year
870903,74738J30,2023-09-30,0.008246,2023
899413,74738J30,2024-03-31,10493.231653,2024
899414,74738J30,2024-06-30,0.007535,2024
